In [ ]:
from huggingface_hub import login
login(token="Your API key")
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
device = torch.device("cuda:2")  # or "cpu"
from sparsify import Sae
import json
import numpy as np
import re
from util import *

model_name = 'llama3'
trainkey = 'prompt_with_cheesecake_train'
key = trainkey
word = "cheese"
step = 0
partword = True 

with open('./data_collection/prompt_bank.json', 'r') as file:
    prompt_bank = json.load(file)


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
input_text_train = prompt_bank[trainkey]
input_text_test = prompt_bank[testkey]


trainkey = 'prompt_with_cheesecake_train'
testkey = 'prompt_with_cheesecake_test'
for word in ['cheese','cake','cheesecake']:
    tokenstrain = obtain_sentence_wise_token(tokenizer, input_text_train,device)
    indicestrain = get_word_indices_in_sequence_partword(tokenstrain, word, step = step) # n step predictive of the upcoming sequence

    tokenstest = obtain_sentence_wise_token(tokenizer, input_text_test, device)
    indicestest = get_word_indices_in_sequence_partword(tokenstest, word, step = step) # n step predictive of the upcoming sequence
    
    maximal_neural_activities_train=[]
    maximal_neural_activities_test=[]

    for i in range(1,32): # iterate through layer, no layer 31, and 32, from EleutherAI tho
        concatenated_hidden_state = np.load(f'./hidden_unit_activity/{trainkey}_concatenated_hidden_state_layer_{i}_model={model_name}.npy')
        hidden_state = concatenated_hidden_state[0,:,:]
        hidden_state = torch.from_numpy(hidden_state).float().to(device)
        sae = Sae.load_from_hub("EleutherAI/sae-llama-3-8b-32x", hookpoint=f"layers.{i-1}").to(device)
        encoder_output = sae.encode(hidden_state)
        #### extract the maximally activating SAE neurons 
        max_neuron_index, max_avg_value = find_maximal_neuron(indicestrain, encoder_output)
        neural_activity = get_batched_neural_activity(max_neuron_index, encoder_output)
        maximal_neural_activities_train.append(neural_activity)

        # load_test, record the activity of the neuron found with maximal activity in the training data 
        concatenated_hidden_state = np.load(f'./hidden_unit_activity/{testkey}_concatenated_hidden_state_layer_{i}_model={model_name}.npy')
        hidden_state = concatenated_hidden_state[0,:,:]
        hidden_state = torch.from_numpy(hidden_state).float().to(device)
        encoder_output = sae.encode(hidden_state)
        neural_activity = get_batched_neural_activity(max_neuron_index, encoder_output)
        maximal_neural_activities_test.append(neural_activity)
        del sae
        torch.cuda.empty_cache()
    
    combined_train = np.stack(maximal_neural_activities_train,axis=0)
    combined_test = np.stack(maximal_neural_activities_test,axis=0)

    # np.save(f"/shared-network/swu/code/experiment_data/sae_key={trainkey}_word={word}.npy", combined_train)
    # np.save(f"/shared-network/swu/code/experiment_data/sae_key={testkey}_word={word}.npy", combined_test)

In [ ]:
from huggingface_hub import login
login(token="Your API TOKEN")
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
device = torch.device("cuda:2")  # or "cpu"
from sparsify import Sae
import json
import numpy as np
import re
from util import *

model_name = 'sae'
step = 0
partword = True 

trainkey = 'prompt_with_cheesecake_train'
testkey = 'prompt_with_cheesecake_test'
with open('./data_collection/prompt_bank.json', 'r') as file:
    prompt_bank = json.load(file)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

input_text = prompt_bank[trainkey]

experiment_data = []
neural_chunk_dictionary = {}
for word in ['cheese','cake','cheesecake']:
    neural_chunk_dictionary[word] = {}
    neural_chunk_dictionary[word][step] = {}
    neural_chunk_dictionary[word][step]['layer'] = {}
    for layer in range(0,32):
        neural_chunk_dictionary[word][step]['layer'][layer] = {}
        neural_chunk_dictionary[word][step]['layer'][layer]['dev_threshold'] = {}

for word in ['cheese','cake','cheesecake']:
    input_text = prompt_bank[trainkey]

    if trainkey == 'austen-emma.txt':tokens = get_token_austin(tokenizer, input_text,device)
    else:tokens = obtain_sentence_wise_token(tokenizer, input_text,device)
    indices = get_word_indices_in_sequence(tokens, word, step = step) # n step predictive of the upcoming sequence
    if partword: indices = get_word_indices_in_sequence_partword(tokens, word, step = step) # n step predictive of the upcoming sequence
    combined = np.load(f"./sae_key={trainkey}_word={word}.npy")

    layers, best_accs, best_tps, best_fps = [], [], [], []
    for layer in range(0,combined.shape[0]):
        hidden_activity = combined[layer, :]
        n_training = len(indices)
        T_range = np.unique(hidden_activity)
        T_range.sort()  # sorts in-place from smallest to largest
        max_acc = 0
        max_tp = 0
        accs, tps, fps = [], [], []
        for dev in T_range: 
            TP_rate, FP_rate, accuracy,_ = get_TP_FP_sae(hidden_activity, indices, tokens, dev)
            # print(TP_rate, FP_rate, dev, accuracy)
            if accuracy>max_acc:
                max_acc = accuracy
                best_threshold = dev 
                neural_chunk_dictionary[word][step]['layer'][layer]['dev_threshold'] = dev
            for lst, value in zip([accs, tps, fps], [accuracy, TP_rate, FP_rate]):
                lst.append(value)
            if TP_rate > max_tp:max_tp = TP_rate
        #print('------------------')
        TP_rate, FP_rate, accuracy,_ = get_TP_FP_sae(hidden_activity, indices, tokens, best_threshold)
        # print('------------------')
    
        temp = []
        for i in range(0, len(tps)):
            if tps[i]>=max_tp:temp.append(fps[i])
        fps_sorted, tps_sorted = zip(*sorted(zip(fps, tps)))
        fps_sorted = fps_sorted + (1,)
        tps_sorted = tps_sorted + (1,)
        auc = np.trapezoid(tps_sorted, fps_sorted)
        
        layers.append(layer)
        best_accs.append(accuracy)
        best_tps.append(TP_rate)
        best_fps.append(FP_rate)
        print(TP_rate, best_tps)
        experiment_data.append({'Model': model_name, 'Word': word ,'step': step, 'TP': TP_rate, 'FP': FP_rate,'n_training': n_training, 'Layer': layer, 'Training': True, 'Test': False})
    plot_decoding_performance(layers, best_tps, best_fps, step = step, word = word, testdata = False,model_name = model_name)
    
    # with open(f"./neural_chunk_dictionary/neural_chunk_dictionary_sae.pkl", "wb") as file:
    #     pickle.dump(neural_chunk_dictionary, file)
    ############### test performance evaluation ##############
    input_text = prompt_bank[testkey]
    if testkey == 'austen-persuasion.txt':tokens = get_token_austin(tokenizer, input_text,device)
    else:tokens = obtain_sentence_wise_token(tokenizer, input_text,device)

    indices = get_word_indices_in_sequence(tokens, word, step = step)
    if partword: indices = get_word_indices_in_sequence_partword(tokens, word, step = step) # n step predictive of the upcoming sequence

    combined = np.load(f"./sae_key={testkey}_word={word}.npy")
    assert word in neural_chunk_dictionary 
    layers, accs, tps, fps = [], [], [], []
    for layer in range(0,combined.shape[0]): # 
        hidden_activity = combined[layer, :] # Load the hidden state from the saved file
        best_threshold = neural_chunk_dictionary[word][step]['layer'][layer]['dev_threshold']
        TP_rate, FP_rate, accuracy,_ = get_TP_FP_sae(hidden_activity,indices,tokens, best_threshold)
        print(TP_rate, FP_rate, accuracy)
        experiment_data.append({'Model': model_name, 'Word': word ,'step': step, 'TP': TP_rate, 'FP': FP_rate,'n_training': n_training, 'Layer': layer, 'Training': False, 'Test': True})
        for lst, value in zip([accs, tps, fps, layers], [accuracy, TP_rate, FP_rate, layer]):
            lst.append(value)

        layers.append(layer)
        accs.append(accuracy)
        tps.append(TP_rate)
        fps.append(FP_rate)
    
    plot_decoding_performance(layers, tps, fps, word = word, step = step, testdata = True, model_name = model_name)


In [27]:
maxsaedata = np.load("./sae_train.npy")

In [ ]:
# we take the activity of the maximally activity neuron when cheese is present across the batch, 
# and we use the activity of this neuron to determine the prescence of cheese 

In [92]:
for layer in range(0,combined.shape[0]):
    hidden_activity = combined[layer, :]
    T_range = np.unique(hidden_activity)
    T_range.sort()  # sorts in-place from smallest to largest
    # choose the best deviation threshold 
    max_acc = 0
    max_tp = 0
    layers, accs, tps, fps = [], [], [], []
    
    for dev in T_range: 
        TP_rate, FP_rate, accuracy,_ = get_TP_FP(hidden_activity, indices, tokens, dev)
        # print(TP_rate, FP_rate, dev, accuracy)
        if accuracy>max_acc:
            max_acc = accuracy
            best_threshold = dev 
            neural_chunk_dictionary[word][step]['layer'][layer]['dev_threshold'] = dev
        
        for lst, value in zip([accs, tps, fps, layers], [accuracy, TP_rate, FP_rate, layer]):
            lst.append(value)
        if TP_rate > max_tp:max_tp = TP_rate
    #print('------------------')
    TP_rate, FP_rate, accuracy,_ = get_TP_FP(hidden_activity, indices, tokens, best_threshold)
    # print(TP_rate, FP_rate, accuracy, best_threshold)
    # print('------------------')

    temp = []
    for i in range(0, len(tps)):
        if tps[i]>=max_tp:temp.append(fps[i])
    
    fps_sorted, tps_sorted = zip(*sorted(zip(fps, tps)))
    fps_sorted = fps_sorted + (1,)
    tps_sorted = tps_sorted + (1,)
    auc = np.trapezoid(tps_sorted, fps_sorted)
    

with open(f"./neural_chunk_dictionary_sae.pkl", "wb") as file:
    pickle.dump(neural_chunk_dictionary, file)


In [28]:
input_text

'Cheese is one of the most versatile ingredients in the culinary world, and cheese can be used in everything from savory dishes to desserts. Cheese lovers often enjoy pairing cheese with crackers, wine, or fruit, but cheese also shines in baking. Cake, on the other hand, is the quintessential dessert, with cake being a staple at celebrations. Cake comes in many forms, such as chocolate cake, vanilla cake, or even carrot cake. However, when you bring cheese and cake together to create cheesecake, a magical transformation happens. Cheesecake is a dessert like no other, with cheesecake offering the creaminess of cheese and the sweetness of cake in perfect harmony. Cheesecake can be topped with fruits like strawberries or blueberries, or cheesecake can be flavored with chocolate or caramel. Some people prefer classic cheesecake, while others enjoy a more decadent cheesecake loaded with toppings. Regardless of the variation, cheesecake remains one of the most beloved desserts worldwide. The

In [ ]:
# record neural activity of the top 1 neuron in each layer, responding to the tokenized text 
# then use the activity to classify input sequence

In [ ]:
model_names = ['llama3']
words = ['cheese', 'cake','cheesecake'] # 'cheese','cake',
trainkey = 'prompt_with_cheesecake_train'
testkey = 'prompt_with_cheesecake_test'

for model_name in model_names:
    for word in words:
        decode_chunks_all_models(word, trainkey, testkey, prompt_bank,device, tokenizer, model_name=model_name, step = step, n_training = 5, partword = True)     
        # input_text = prompt_bank[trainkey]
        # tokens = obtain_sentence_wise_token(tokenizer, input_text, device)

In [ ]:
# extract the activity of the biggest sae neuron and store them in a file like this 
hidden_state = np.load(f'./hidden_unit_activity/{trainkey}_concatenated_hidden_state_layer_{layer}_model={model_name}.npy')
# after storing them in a separate file, then use the plotting code to process them 